# DQN From Scratch: Teaching a Neural Network to Play Games

Welcome to DQN - the algorithm that taught computers to play Atari games at superhuman level! This notebook will build a complete DQN from scratch with clear explanations.

## What You'll Learn

By the end of this notebook, you'll understand:
- Why we need neural networks for Q-learning (the scaling problem)
- What DQN is and how it works (with a brain upgrade analogy!)
- Experience replay (the memory bank trick)
- Target networks (the slow twin trick)
- How to implement DQN from scratch in PyTorch
- Train an agent to balance a pole!

**Prerequisites:** Notebooks in `classic-algorithms/` (Q-learning basics)

**Time:** ~45 minutes

---
## The Big Picture: Why Deep Q-Learning?

### The Problem with Tables

Remember Q-learning? We stored Q-values in a table:

```
    ┌────────────────────────────────────────────────────────┐
    │                  THE TABLE PROBLEM                      │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Q-Table for a 4x4 Grid:   Easy! Only 16 states        │
    │  ┌─────────────────────┐                               │
    │  │ State │ UP │ DN │ ...│                               │
    │  ├─────────────────────┤                               │
    │  │ (0,0) │ 2.3│ 1.2│ ...│                               │
    │  │ (0,1) │ 3.1│ 2.5│ ...│                               │
    │  └─────────────────────┘                               │
    │                                                         │
    │  Q-Table for Atari:      IMPOSSIBLE!                   │
    │  • Screen: 210 × 160 pixels × 3 colors                 │
    │  • Each pixel: 256 possible values                     │
    │  • Total states: 256^(210×160×3) ≈ ∞                   │
    │                                                         │
    │  We can't store a table with infinite rows!            │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

### The Solution: Neural Networks!

Instead of memorizing every state, we **learn patterns**:

```
    ┌────────────────────────────────────────────────────────┐
    │            FROM TABLE TO NEURAL NETWORK                 │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Q-TABLE (Memorization):                               │
    │    state → lookup in table → Q-values                  │
    │    "I've seen this exact state before"                 │
    │                                                         │
    │  Q-NETWORK (Generalization):                           │
    │    state → neural network → Q-values                   │
    │    "This looks SIMILAR to states I've seen"            │
    │                                                         │
    │  ┌─────┐      ┌─────────────┐      ┌─────────┐         │
    │  │State│ ───> │   Neural    │ ───> │Q-values │         │
    │  │Image│      │   Network   │      │for each │         │
    │  └─────┘      └─────────────┘      │ action  │         │
    │                                    └─────────┘         │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

---
## The Brain Upgrade Analogy

Think of DQN as **upgrading your brain**:

```
    ┌────────────────────────────────────────────────────────┐
    │            THE BRAIN UPGRADE ANALOGY                    │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  TABULAR Q-LEARNING = Flashcard Memory                 │
    │    📝 "If I see THIS exact problem, the answer is X"   │
    │    • Must see every problem to know the answer         │
    │    • Can't handle new problems                         │
    │    • Memory fills up quickly                           │
    │                                                         │
    │  DQN = Pattern Recognition Brain                       │
    │    🧠 "Problems with THESE features usually need X"     │
    │    • Learns general patterns                           │
    │    • Can handle new situations                         │
    │    • Much more efficient!                              │
    │                                                         │
    │  Example: Recognizing faces                            │
    │    Flashcard: Store every possible face image          │
    │    Brain: Learn "eyes go here, nose goes here"         │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch, Circle
from IPython.display import clear_output

# Try to import gymnasium, fall back to gym
try:
    import gymnasium as gym
except ImportError:
    import gym

# Visualize the DQN architecture
fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')

# Title
ax.text(7, 9.5, 'Deep Q-Network (DQN) Architecture', 
        ha='center', fontsize=18, fontweight='bold')

# Input: State
state_box = FancyBboxPatch((0.5, 3.5), 2.5, 3, boxstyle="round,pad=0.1",
                            facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(state_box)
ax.text(1.75, 5.5, 'STATE', ha='center', fontsize=14, fontweight='bold', color='#1976d2')
ax.text(1.75, 4.8, '(observation)', ha='center', fontsize=10, color='#1976d2')
ax.text(1.75, 4.2, 'e.g., cart pos,\npole angle...', ha='center', fontsize=9, color='#666')

# Neural Network
nn_box = FancyBboxPatch((4.5, 2.5), 5, 5, boxstyle="round,pad=0.1",
                         facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(nn_box)
ax.text(7, 7, 'NEURAL NETWORK', ha='center', fontsize=14, fontweight='bold', color='#f57c00')

# Draw neurons
layers = [
    [(5.5, 5.5), (5.5, 4.5), (5.5, 3.5)],  # Input layer
    [(7, 5.8), (7, 5), (7, 4.2), (7, 3.4)],  # Hidden layer
    [(8.5, 5.5), (8.5, 4.5), (8.5, 3.5)]   # Output layer
]

colors = ['#bbdefb', '#ffcc80', '#c8e6c9']
for layer_idx, layer in enumerate(layers):
    for (x, y) in layer:
        circle = Circle((x, y), 0.25, facecolor=colors[layer_idx], 
                        edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)

# Draw connections (simplified)
for (x1, y1) in layers[0]:
    for (x2, y2) in layers[1]:
        ax.plot([x1+0.25, x2-0.25], [y1, y2], 'gray', linewidth=0.5, alpha=0.5)
for (x1, y1) in layers[1]:
    for (x2, y2) in layers[2]:
        ax.plot([x1+0.25, x2-0.25], [y1, y2], 'gray', linewidth=0.5, alpha=0.5)

ax.text(5.5, 2.8, 'Input', ha='center', fontsize=9)
ax.text(7, 2.8, 'Hidden', ha='center', fontsize=9)
ax.text(8.5, 2.8, 'Output', ha='center', fontsize=9)

# Output: Q-values
output_box = FancyBboxPatch((10.5, 3), 3, 4, boxstyle="round,pad=0.1",
                             facecolor='#e8f5e9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(output_box)
ax.text(12, 6.5, 'Q-VALUES', ha='center', fontsize=14, fontweight='bold', color='#388e3c')
ax.text(12, 5.6, 'Q(s, left) = 2.3', ha='center', fontsize=11, color='#666')
ax.text(12, 4.8, 'Q(s, right) = 5.1 ←', ha='center', fontsize=11, color='#388e3c', fontweight='bold')
ax.text(12, 3.8, 'Best action: RIGHT', ha='center', fontsize=10, color='#388e3c', style='italic')

# Arrows
ax.annotate('', xy=(4.3, 5), xytext=(3.1, 5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#1976d2'))
ax.annotate('', xy=(10.3, 5), xytext=(9.7, 5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#388e3c'))

# Bottom explanation
ax.text(7, 1.5, 'The neural network takes a state as input and outputs Q-values for ALL actions',
        ha='center', fontsize=11, color='#333')
ax.text(7, 0.8, 'Action selection: Pick the action with the highest Q-value!',
        ha='center', fontsize=11, color='#333', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("KEY INSIGHT: From Table to Function")
print("="*60)
print("\nQ-Table:   Q: (state, action) → one value (lookup)")
print("Q-Network: Q: state → [value for each action] (computation)")
print("\nThe neural network LEARNS the mapping through training!")
print("="*60)

---
## The CartPole Environment: Our Test Game

We'll train our DQN to play **CartPole** - a classic control problem:

```
    ┌────────────────────────────────────────────────────────┐
    │                    CARTPOLE GAME                        │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │                      ╱╲                                 │
    │                     ╱  ╲   ← Pole (keep it balanced!)   │
    │                    ╱    ╲                               │
    │                   ╱      ╲                              │
    │              ┌───────────┐                              │
    │              │   Cart    │                              │
    │              └───────────┘                              │
    │    ════════════════════════════════════                 │
    │              ← LEFT    RIGHT →                          │
    │                                                         │
    │  GOAL: Keep the pole balanced as long as possible!     │
    │                                                         │
    │  STATE (what the agent sees):                          │
    │    1. Cart position (-2.4 to 2.4)                      │
    │    2. Cart velocity                                    │
    │    3. Pole angle (-12° to 12°)                         │
    │    4. Pole angular velocity                            │
    │                                                         │
    │  ACTIONS (what the agent can do):                      │
    │    0 = Push cart LEFT                                  │
    │    1 = Push cart RIGHT                                 │
    │                                                         │
    │  REWARD: +1 for each timestep the pole stays up       │
    │                                                         │
    │  SOLVED: Average reward ≥ 195 over 100 episodes        │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

In [ ]:
# Create and explore the CartPole environment
env = gym.make('CartPole-v1')

print("CARTPOLE ENVIRONMENT")
print("="*50)
print(f"\nObservation space: {env.observation_space}")
print(f"  Shape: {env.observation_space.shape}")
print(f"  Meaning: [cart_pos, cart_vel, pole_angle, pole_vel]")
print(f"\nAction space: {env.action_space}")
print(f"  0 = Push LEFT")
print(f"  1 = Push RIGHT")

# Show a sample observation
state, _ = env.reset()
print(f"\nSample starting state: {state}")
print(f"  Cart position:  {state[0]:.4f}")
print(f"  Cart velocity:  {state[1]:.4f}")
print(f"  Pole angle:     {state[2]:.4f} rad ({np.degrees(state[2]):.2f}°)")
print(f"  Pole velocity:  {state[3]:.4f}")

env.close()
print("\n" + "="*50)

---
## Step 1: Building the Q-Network

Our Q-Network is a simple feedforward neural network:

```
    ┌────────────────────────────────────────────────────────┐
    │               Q-NETWORK ARCHITECTURE                    │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Input Layer (4 neurons):                              │
    │    [cart_pos, cart_vel, pole_angle, pole_vel]          │
    │                         ↓                               │
    │  Hidden Layer 1 (128 neurons) + ReLU:                  │
    │    Learn intermediate features                         │
    │                         ↓                               │
    │  Hidden Layer 2 (128 neurons) + ReLU:                  │
    │    Learn more complex patterns                         │
    │                         ↓                               │
    │  Output Layer (2 neurons):                             │
    │    [Q(s, LEFT), Q(s, RIGHT)]                           │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

In [ ]:
class QNetwork(nn.Module):
    """
    Deep Q-Network: A neural network that estimates Q-values.
    
    Input: State (e.g., 4 values for CartPole)
    Output: Q-value for EACH action (e.g., 2 values for LEFT and RIGHT)
    
    Architecture:
        state (4) → Linear(128) → ReLU → Linear(128) → ReLU → Linear(2)
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        
        # Store dimensions for later use
        self.state_dim = state_dim
        self.action_dim = action_dim
        
        # Build the network
        self.network = nn.Sequential(
            # Layer 1: Input → Hidden
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            
            # Layer 2: Hidden → Hidden
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            
            # Layer 3: Hidden → Output (Q-values)
            nn.Linear(hidden_dim, action_dim)
            # No activation! Q-values can be any number
        )
    
    def forward(self, x):
        """
        Forward pass: state → Q-values.
        
        Args:
            x: Batch of states, shape (batch_size, state_dim)
        
        Returns:
            Q-values for each action, shape (batch_size, action_dim)
        """
        return self.network(x)
    
    def get_action(self, state, epsilon=0.0):
        """
        Choose an action using epsilon-greedy policy.
        
        Args:
            state: Current state (numpy array)
            epsilon: Probability of random action (exploration)
        
        Returns:
            Action to take (integer)
        """
        # Exploration: Random action
        if random.random() < epsilon:
            return random.randint(0, self.action_dim - 1)
        
        # Exploitation: Best action based on Q-values
        with torch.no_grad():  # No gradients needed for action selection
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Add batch dimension
            q_values = self.forward(state_tensor)
            return q_values.argmax().item()  # Return action with highest Q-value


# Create and test the network
print("Q-NETWORK DEMONSTRATION")
print("="*60)

# Create network for CartPole (4 inputs, 2 outputs)
q_net = QNetwork(state_dim=4, action_dim=2)
print(f"\nNetwork architecture:")
print(q_net)

# Test with a sample state
sample_state = np.array([0.01, 0.02, -0.03, 0.04])
state_tensor = torch.FloatTensor(sample_state).unsqueeze(0)

with torch.no_grad():
    q_values = q_net(state_tensor)

print(f"\nSample state: {sample_state}")
print(f"Q-values output: {q_values.numpy()[0]}")
print(f"  Q(s, LEFT):  {q_values[0, 0]:.4f}")
print(f"  Q(s, RIGHT): {q_values[0, 1]:.4f}")
print(f"  Best action: {'LEFT' if q_values.argmax() == 0 else 'RIGHT'}")

# Count parameters
total_params = sum(p.numel() for p in q_net.parameters())
print(f"\nTotal parameters: {total_params:,}")
print("(This is much smaller than storing a table for continuous states!)")
print("="*60)

---
## Step 2: Experience Replay - The Memory Bank Trick

### The Problem: Correlated Data

When learning in sequence, consecutive experiences are very similar:

```
    ┌────────────────────────────────────────────────────────┐
    │            THE PROBLEM WITH ONLINE LEARNING             │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Consecutive experiences are CORRELATED:               │
    │                                                         │
    │  Time 1: Cart at position 0.1, pole angle 0.05        │
    │  Time 2: Cart at position 0.12, pole angle 0.06       │
    │  Time 3: Cart at position 0.14, pole angle 0.07       │
    │                                                         │
    │  These are almost IDENTICAL! If we train on these     │
    │  in order, the network overfits to this one situation │
    │  and forgets everything else!                          │
    │                                                         │
    │  Analogy: Studying for a test                          │
    │    Bad: Read chapter 5, chapter 5, chapter 5... (×100) │
    │    Good: Random mix of all chapters!                   │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

### The Solution: Memory Bank

```
    ┌────────────────────────────────────────────────────────┐
    │            EXPERIENCE REPLAY: THE MEMORY BANK           │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  1. STORE experiences in a memory bank (buffer)        │
    │                                                         │
    │     Memory Bank (holds thousands of experiences)       │
    │     ┌─────────────────────────────────────────┐        │
    │     │ (s₁,a₁,r₁,s'₁) │ (s₂,a₂,r₂,s'₂) │ ... │        │
    │     │ (s₃,a₃,r₃,s'₃) │ (s₄,a₄,r₄,s'₄) │ ... │        │
    │     │ (s₅,a₅,r₅,s'₅) │ (s₆,a₆,r₆,s'₆) │ ... │        │
    │     └─────────────────────────────────────────┘        │
    │                                                         │
    │  2. SAMPLE random mini-batches for training            │
    │                                                         │
    │     Random Sample:                                     │
    │     [exp_42, exp_1337, exp_99, exp_500, ...]          │
    │                                                         │
    │     These are from DIFFERENT times and situations!    │
    │     Much more diverse training data!                   │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

In [ ]:
class ReplayBuffer:
    """
    Experience Replay Buffer - The Memory Bank!
    
    Stores transitions (s, a, r, s', done) and allows random sampling.
    Uses a deque (double-ended queue) that automatically removes old
    experiences when capacity is reached.
    
    Analogy: A library with limited shelf space.
    - New books go on the shelf
    - Old books are removed when shelf is full
    - You can randomly pick any book to read!
    """
    
    def __init__(self, capacity=10000):
        """
        Args:
            capacity: Maximum number of experiences to store.
                      Once full, oldest experiences are removed.
        """
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
    
    def push(self, state, action, reward, next_state, done):
        """
        Store a new experience in the buffer.
        
        Args:
            state: The state before the action
            action: The action taken
            reward: The reward received
            next_state: The resulting state
            done: Whether the episode ended
        """
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        """
        Randomly sample a batch of experiences.
        
        This is the key insight! Random sampling breaks correlations!
        
        Returns:
            Tuple of tensors: (states, actions, rewards, next_states, dones)
        """
        # Randomly sample batch_size experiences
        batch = random.sample(self.buffer, batch_size)
        
        # Unzip the batch into separate lists
        states, actions, rewards, next_states, dones = zip(*batch)
        
        # Convert to PyTorch tensors
        return (
            torch.FloatTensor(np.array(states)),
            torch.LongTensor(actions),
            torch.FloatTensor(rewards),
            torch.FloatTensor(np.array(next_states)),
            torch.FloatTensor(dones)
        )
    
    def __len__(self):
        return len(self.buffer)


# Demonstrate the replay buffer
print("REPLAY BUFFER DEMONSTRATION")
print("="*60)

buffer = ReplayBuffer(capacity=1000)

# Add some fake experiences
print("\nAdding 100 experiences to the buffer...")
for i in range(100):
    state = np.random.randn(4)
    action = random.randint(0, 1)
    reward = 1.0
    next_state = np.random.randn(4)
    done = False
    buffer.push(state, action, reward, next_state, done)

print(f"Buffer size: {len(buffer)} / {buffer.capacity}")

# Sample a batch
print("\nSampling a batch of 5 experiences...")
states, actions, rewards, next_states, dones = buffer.sample(5)

print(f"States shape: {states.shape}")
print(f"Actions: {actions.tolist()}")
print(f"Rewards: {rewards.tolist()}")

print("\n" + "-"*60)
print("KEY BENEFITS OF EXPERIENCE REPLAY:")
print("-"*60)
print("1. BREAKS CORRELATION: Random sampling removes temporal patterns")
print("2. REUSES DATA: Same experience can be sampled multiple times")
print("3. STABILIZES TRAINING: More diverse mini-batches")
print("="*60)

---
## Step 3: Target Network - The Slow Twin Trick

### The Problem: Moving Target

```
    ┌────────────────────────────────────────────────────────┐
    │            THE MOVING TARGET PROBLEM                    │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Q-learning update:                                    │
    │    Q(s,a) ← Q(s,a) + α[r + γ·max Q(s',a') - Q(s,a)]   │
    │                           ↑                            │
    │                     This is our TARGET                 │
    │                                                         │
    │  THE PROBLEM:                                          │
    │    We use the SAME network for:                        │
    │    1. Predicting Q(s,a)  (what we're training)        │
    │    2. Computing target max Q(s',a')                    │
    │                                                         │
    │    When we update the network, BOTH change!            │
    │    It's like chasing a target that keeps moving!       │
    │                                                         │
    │  Analogy: Trying to hit a moving target               │
    │    Every time you aim and shoot, the target moves!    │
    │    Very hard to converge!                              │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

### The Solution: A Slow Twin

```
    ┌────────────────────────────────────────────────────────┐
    │            TARGET NETWORK: THE SLOW TWIN                │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Use TWO networks:                                     │
    │                                                         │
    │  ┌─────────────────┐    ┌─────────────────┐            │
    │  │  Q-Network      │    │ Target Network   │            │
    │  │  (Main/Online)  │    │ (Slow Twin)      │            │
    │  ├─────────────────┤    ├─────────────────┤            │
    │  │ Updated EVERY   │    │ Updated every   │            │
    │  │ training step   │    │ N steps (slowly) │            │
    │  │                 │    │                 │            │
    │  │ Used for:       │    │ Used for:       │            │
    │  │ - Action select │    │ - Computing     │            │
    │  │ - Q(s,a)        │    │   TARGET values │            │
    │  └─────────────────┘    └─────────────────┘            │
    │           ↑                     ↑                       │
    │           │         Copy every N steps                 │
    │           └─────────────────────┘                       │
    │                                                         │
    │  The target network is a FROZEN COPY that only updates │
    │  periodically, giving a STABLE target to chase!        │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the target network concept
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Without target network (unstable)
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 8)
ax1.axis('off')
ax1.set_title('WITHOUT Target Network\n(Unstable!)', fontsize=14, fontweight='bold', color='#d32f2f')

# Single network
net_box = FancyBboxPatch((3, 3), 4, 3, boxstyle="round,pad=0.1",
                          facecolor='#ffcdd2', edgecolor='#d32f2f', linewidth=2)
ax1.add_patch(net_box)
ax1.text(5, 5, 'Q-Network', ha='center', fontsize=12, fontweight='bold')
ax1.text(5, 4.2, 'Predicts Q(s,a)', ha='center', fontsize=10)
ax1.text(5, 3.5, 'AND targets!', ha='center', fontsize=10, color='#d32f2f')

# Arrows showing circular dependency
ax1.annotate('', xy=(7.2, 5.5), xytext=(7.2, 2.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f', 
                            connectionstyle='arc3,rad=0.3'))
ax1.annotate('', xy=(2.8, 2.5), xytext=(2.8, 5.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f',
                            connectionstyle='arc3,rad=0.3'))

ax1.text(5, 1.5, 'Updates change both prediction AND target!', 
         ha='center', fontsize=10, color='#d32f2f')
ax1.text(5, 0.8, '→ Chasing a moving target = UNSTABLE', 
         ha='center', fontsize=10, color='#d32f2f', fontweight='bold')

# Right: With target network (stable)
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 8)
ax2.axis('off')
ax2.set_title('WITH Target Network\n(Stable!)', fontsize=14, fontweight='bold', color='#388e3c')

# Q-Network
q_box = FancyBboxPatch((0.5, 3), 3.5, 3, boxstyle="round,pad=0.1",
                        facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
ax2.add_patch(q_box)
ax2.text(2.25, 5, 'Q-Network', ha='center', fontsize=11, fontweight='bold', color='#1976d2')
ax2.text(2.25, 4.2, 'Updates every', ha='center', fontsize=9)
ax2.text(2.25, 3.6, 'training step', ha='center', fontsize=9)

# Target Network
target_box = FancyBboxPatch((6, 3), 3.5, 3, boxstyle="round,pad=0.1",
                             facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=2)
ax2.add_patch(target_box)
ax2.text(7.75, 5, 'Target Network', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
ax2.text(7.75, 4.2, 'Updates every', ha='center', fontsize=9)
ax2.text(7.75, 3.6, 'N steps (slow)', ha='center', fontsize=9)

# Copy arrow
ax2.annotate('', xy=(5.8, 4.5), xytext=(4.2, 4.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666', linestyle='--'))
ax2.text(5, 5.2, 'Copy every N steps', ha='center', fontsize=9, color='#666')

ax2.text(5, 1.5, 'Target stays FIXED between copies!', 
         ha='center', fontsize=10, color='#388e3c')
ax2.text(5, 0.8, '→ Stable target = STABLE training', 
         ha='center', fontsize=10, color='#388e3c', fontweight='bold')

plt.tight_layout()
plt.show()

---
## Step 4: The Complete DQN Agent

Now let's put it all together!

In [ ]:
class DQNAgent:
    """
    Complete DQN Agent with:
    - Q-Network (for predictions)
    - Target Network (for stable targets)
    - Experience Replay (for breaking correlations)
    
    This is the algorithm that DeepMind used to play Atari games!
    """
    
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99,
                 buffer_size=10000, batch_size=64, target_update=100):
        """
        Args:
            state_dim: Size of state observation
            action_dim: Number of possible actions
            lr: Learning rate for optimizer
            gamma: Discount factor for future rewards
            buffer_size: Capacity of replay buffer
            batch_size: Number of samples per training batch
            target_update: How often to update target network
        """
        self.action_dim = action_dim
        self.gamma = gamma
        self.batch_size = batch_size
        self.target_update = target_update
        
        # ========================================
        # CREATE TWO NETWORKS
        # ========================================
        # Main Q-Network (updated every step)
        self.q_network = QNetwork(state_dim, action_dim)
        
        # Target Network (updated slowly)
        self.target_network = QNetwork(state_dim, action_dim)
        # Start with same weights as Q-network
        self.target_network.load_state_dict(self.q_network.state_dict())
        
        # ========================================
        # OPTIMIZER AND REPLAY BUFFER
        # ========================================
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.buffer = ReplayBuffer(buffer_size)
        
        # Step counter for target network updates
        self.steps = 0
    
    def act(self, state, epsilon=0.0):
        """Choose action using epsilon-greedy policy."""
        return self.q_network.get_action(state, epsilon)
    
    def store(self, state, action, reward, next_state, done):
        """Store experience in replay buffer."""
        self.buffer.push(state, action, reward, next_state, done)
    
    def update(self):
        """
        Perform one training step.
        
        This is where the learning happens!
        """
        # Need enough experiences to sample a batch
        if len(self.buffer) < self.batch_size:
            return None
        
        # ========================================
        # STEP 1: Sample a batch from memory
        # ========================================
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        # ========================================
        # STEP 2: Compute current Q-values
        # ========================================
        # Q(s, a) for the actions we actually took
        q_values = self.q_network(states)  # Shape: (batch, num_actions)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze()  # Q(s, a)
        
        # ========================================
        # STEP 3: Compute target Q-values
        # ========================================
        # Use TARGET network for stable targets!
        with torch.no_grad():  # Don't compute gradients for targets
            next_q_values = self.target_network(next_states)
            max_next_q = next_q_values.max(dim=1)[0]  # max Q(s', a')
            
            # Target: r + γ * max Q(s', a') * (1 - done)
            # The (1 - done) zeroes out future value for terminal states
            targets = rewards + self.gamma * max_next_q * (1 - dones)
        
        # ========================================
        # STEP 4: Compute loss and update
        # ========================================
        # Loss: Mean squared error between prediction and target
        loss = nn.functional.mse_loss(q_values, targets)
        
        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # ========================================
        # STEP 5: Update target network periodically
        # ========================================
        self.steps += 1
        if self.steps % self.target_update == 0:
            # Copy weights from Q-network to target network
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()


print("DQN AGENT COMPONENTS")
print("="*60)
print("""
┌──────────────────────────────────────────────────────────┐
│                    DQN AGENT SUMMARY                     │
├──────────────────────────────────────────────────────────┤
│                                                          │
│  1. Q-NETWORK (Main)                                    │
│     → Predicts Q-values for action selection            │
│     → Updated every training step                       │
│                                                          │
│  2. TARGET NETWORK (Slow Twin)                          │
│     → Provides stable targets for training              │
│     → Copied from Q-network every N steps               │
│                                                          │
│  3. REPLAY BUFFER (Memory Bank)                         │
│     → Stores past experiences                           │
│     → Enables random sampling for training              │
│                                                          │
│  TRAINING LOOP:                                         │
│     1. Act in environment (epsilon-greedy)              │
│     2. Store experience in buffer                       │
│     3. Sample random batch from buffer                  │
│     4. Compute loss: (Q(s,a) - target)²                 │
│     5. Update Q-network with gradient descent           │
│     6. Periodically copy to target network              │
│                                                          │
└──────────────────────────────────────────────────────────┘
""")
print("="*60)

---
## Step 5: Training the DQN Agent

Now let's train our agent to balance the pole!

In [ ]:
def train_dqn(env_name='CartPole-v1', n_episodes=500, 
              epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995,
              verbose=True):
    """
    Train a DQN agent on the specified environment.
    
    Args:
        env_name: Gymnasium environment name
        n_episodes: Number of episodes to train
        epsilon_start: Starting exploration rate
        epsilon_end: Minimum exploration rate
        epsilon_decay: How fast to reduce exploration
        verbose: Whether to print progress
    
    Returns:
        agent: Trained DQN agent
        rewards_history: List of episode rewards
        losses_history: List of training losses
    """
    # Create environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    # Create agent
    agent = DQNAgent(state_dim, action_dim)
    
    # Training tracking
    rewards_history = []
    losses_history = []
    epsilon = epsilon_start
    
    if verbose:
        print("TRAINING DQN ON CARTPOLE")
        print("="*60)
        print(f"Episodes: {n_episodes}")
        print(f"Epsilon: {epsilon_start} → {epsilon_end}")
        print("="*60)
    
    for episode in range(n_episodes):
        # Reset environment
        state, _ = env.reset()
        total_reward = 0
        episode_losses = []
        
        for t in range(500):  # Max 500 steps per episode
            # ========================================
            # 1. Choose action (epsilon-greedy)
            # ========================================
            action = agent.act(state, epsilon)
            
            # ========================================
            # 2. Take action in environment
            # ========================================
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # ========================================
            # 3. Store experience
            # ========================================
            agent.store(state, action, reward, next_state, float(done))
            
            # ========================================
            # 4. Train the agent
            # ========================================
            loss = agent.update()
            if loss is not None:
                episode_losses.append(loss)
            
            # ========================================
            # 5. Move to next state
            # ========================================
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        # Track results
        rewards_history.append(total_reward)
        if episode_losses:
            losses_history.append(np.mean(episode_losses))
        
        # Decay epsilon
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        
        # Print progress
        if verbose and (episode + 1) % 50 == 0:
            avg_reward = np.mean(rewards_history[-50:])
            print(f"Episode {episode+1:4d} | "
                  f"Avg Reward (last 50): {avg_reward:6.1f} | "
                  f"Epsilon: {epsilon:.3f}")
    
    env.close()
    
    if verbose:
        print("="*60)
        print(f"Training complete!")
        final_avg = np.mean(rewards_history[-100:])
        print(f"Final average reward (last 100): {final_avg:.1f}")
        print(f"Solved threshold: 195.0")
        print(f"Status: {'SOLVED!' if final_avg >= 195 else 'Keep training...'}")
        print("="*60)
    
    return agent, rewards_history, losses_history

# Train the agent!
agent, rewards, losses = train_dqn(n_episodes=300)

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Rewards
ax1 = axes[0]
ax1.plot(rewards, alpha=0.3, color='blue', label='Raw')

# Smoothed rewards
window = 20
if len(rewards) >= window:
    smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(rewards)), smoothed, color='blue', 
             linewidth=2, label=f'Smoothed (window={window})')

# Solved threshold
ax1.axhline(y=195, color='green', linestyle='--', linewidth=2, label='Solved (195)')

ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Total Reward', fontsize=12)
ax1.set_title('DQN Learning Curve', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Right: Training loss
ax2 = axes[1]
if losses:
    ax2.plot(losses, alpha=0.5, color='red')
    # Smoothed loss
    if len(losses) >= window:
        smoothed_loss = np.convolve(losses, np.ones(window)/window, mode='valid')
        ax2.plot(range(window-1, len(losses)), smoothed_loss, color='red', linewidth=2)

ax2.set_xlabel('Episode', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('Training Loss', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTRAINING ANALYSIS:")
print("="*60)
print("\n1. REWARDS (left graph):")
print("   - Should increase over time as agent learns")
print("   - Goal: Reach and maintain ≥195 (solved!)")
print("\n2. LOSS (right graph):")
print("   - High initially (random predictions)")
print("   - Should decrease as Q-values become accurate")
print("   - Some noise is normal (stochastic training)")
print("="*60)

---
## Step 6: Watch the Trained Agent Play!

In [ ]:
def evaluate_agent(agent, env_name='CartPole-v1', n_episodes=10):
    """
    Evaluate the trained agent (no exploration, pure exploitation).
    """
    env = gym.make(env_name)
    rewards = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        
        for t in range(500):
            # No exploration (epsilon=0)
            action = agent.act(state, epsilon=0.0)
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            
            if terminated or truncated:
                break
        
        rewards.append(total_reward)
    
    env.close()
    return rewards

# Evaluate the trained agent
print("EVALUATING TRAINED AGENT")
print("="*60)
print("Running 10 episodes with NO exploration (epsilon=0)...\n")

eval_rewards = evaluate_agent(agent, n_episodes=10)

for i, r in enumerate(eval_rewards):
    status = "🎉" if r >= 195 else "😔"
    print(f"Episode {i+1:2d}: {r:5.0f} steps {status}")

print("\n" + "-"*60)
print(f"Average: {np.mean(eval_rewards):.1f} steps")
print(f"Best:    {max(eval_rewards):.0f} steps")
print(f"Worst:   {min(eval_rewards):.0f} steps")
print("\n" + "="*60)

if np.mean(eval_rewards) >= 195:
    print("\n🎉 CONGRATULATIONS! The agent has solved CartPole!")
    print("   It can balance the pole for 195+ steps on average!")
else:
    print("\n📈 The agent is learning but needs more training.")
    print("   Try training for more episodes!")

---
## The Complete DQN Algorithm

Let's summarize the full algorithm:

```
    ┌────────────────────────────────────────────────────────┐
    │                  DQN ALGORITHM                          │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Initialize:                                           │
    │    - Q-network with random weights θ                   │
    │    - Target network with weights θ⁻ = θ                │
    │    - Replay buffer D                                   │
    │                                                         │
    │  For each episode:                                     │
    │    s = initial state                                   │
    │                                                         │
    │    For each step:                                      │
    │      1. Choose a = ε-greedy(Q(s, ·; θ))               │
    │      2. Execute a, observe r, s'                       │
    │      3. Store (s, a, r, s', done) in D                 │
    │      4. Sample mini-batch from D                       │
    │      5. Compute target: y = r + γ·max Q(s', a'; θ⁻)   │
    │      6. Minimize loss: (Q(s, a; θ) - y)²              │
    │      7. Every N steps: θ⁻ ← θ                          │
    │      8. s ← s'                                         │
    │                                                         │
    │    Decay ε                                             │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

---
## Summary: Key Takeaways

### What is DQN?

DQN = Q-Learning + Neural Networks + Two Key Tricks

| Component | Purpose | Analogy |
|-----------|---------|----------|
| Q-Network | Approximate Q-values | Brain that generalizes |
| Experience Replay | Break correlations | Random study from notes |
| Target Network | Stable training targets | Slow-moving goal post |

### Why Each Component Matters

1. **Neural Network instead of Table**
   - Handles continuous/large state spaces
   - Generalizes to unseen states

2. **Experience Replay**
   - Breaks temporal correlation
   - Reuses data efficiently
   - Stabilizes training

3. **Target Network**
   - Provides stable training targets
   - Prevents oscillation/divergence

### Historical Note

DQN was published by DeepMind in 2015 and was the first algorithm to:
- Play Atari games directly from pixels
- Achieve superhuman performance on many games
- Show that deep learning + RL could work together

---
## Test Your Understanding

**1. Why can't we use a Q-table for Atari games?**
<details>
<summary>Click to reveal answer</summary>
The state space is too large! An Atari screen is 210×160×3 pixels with 256 possible values per pixel. This means there are 256^(210×160×3) ≈ 10^300000 possible states - far more than atoms in the universe! We can't store a table that big.
</details>

**2. What problem does experience replay solve?**
<details>
<summary>Click to reveal answer</summary>
Experience replay breaks the correlation between consecutive experiences. Without it, the agent trains on very similar, sequential data which causes it to overfit to recent experiences and forget older ones. Random sampling from the buffer provides more diverse, uncorrelated training data.
</details>

**3. Why do we need a target network?**
<details>
<summary>Click to reveal answer</summary>
Without a target network, both the predictions Q(s,a) and the targets r + γ·max Q(s',a') change every training step since they use the same network. This is like chasing a moving target, causing unstable training. The target network provides fixed targets that only change periodically, stabilizing learning.
</details>

**4. What is epsilon in epsilon-greedy and why does it decay?**
<details>
<summary>Click to reveal answer</summary>
Epsilon is the probability of taking a random action (exploration). It starts high (e.g., 1.0 = 100% random) so the agent explores many states. It decays over time because as the agent learns better Q-values, it should exploit its knowledge more and explore less.
</details>

**5. What does the loss function measure in DQN?**
<details>
<summary>Click to reveal answer</summary>
The loss measures the squared difference between the predicted Q-value Q(s,a) and the target y = r + γ·max Q(s',a'). Minimizing this loss makes the Q-network's predictions more accurate according to the Bellman equation.
</details>

---
## What's Next?

Excellent work! You've implemented DQN from scratch!

In the next notebooks, we'll dive deeper into:
- **Experience Replay** variations (Prioritized Experience Replay)
- **Target Networks** and soft updates
- **DQN Improvements** (Double DQN, Dueling DQN)

**Continue to:** [Notebook 3: Experience Replay](03_experience_replay.ipynb)

---

*DQN was a breakthrough that showed deep learning and RL could work together. You've just implemented the same algorithm that learned to play Atari games at superhuman level!*